In [ ]:
'''
주식거래에 대한 가정
참가자들은 포트폴리오에서 지정해준 순위에 따라 거래기간 동안 거래를 진행한다고 가정
구체적으로는, 전체 2000주 중 상위 200주와 하위 200주를 선정하여 매일 동일 금액 비중으로 거래를 진행한다고 가정
상위 200주에 대해서는 매수 전략(Long)을 취하게 되고, 하위 200주에 대해서는 공매도 전략(Short Selling)을 취함
거래기간 중 매일 종가에 매수와 공매도가 이루어진다고 가정
매수를 통해 보유한 주식은 다음날 종가에 판매한다고 가정
공매도의 상환 역시 다음날 종가에 이루어진다고 가정
결론적으로 참가자들은 거래기간 동안 매일 Long-Short을 반복한다고 가정하고 평가가 진행되게 됨



'''

In [2]:
import pandas as pd
import os


data = pd.read_csv('data/train.csv',encoding='utf8')
data.head()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050


In [3]:
data.shape

(988000, 8)

In [5]:
data.dtypes

일자       int64
종목코드    object
종목명     object
거래량      int64
시가       int64
고가       int64
저가       int64
종가       int64
dtype: object

In [6]:
data.tail()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200
987999,20230530,A238490,힘스,2611843,6410,8220,6300,8220


In [9]:
#기준값 : 일자(202X0530) / 종가
use_col = ['일자','종목코드','종가']
# 2021
bse_data = data[data['일자']==20220530][use_col]
print(bse_data.shape)
bse_data.head()

(2000, 3)


,일자,종목코드,종가
492000,20220530,A060310,3370
492001,20220530,A095570,6960
492002,20220530,A006840,20500
492003,20220530,A054620,11600
492004,20220530,A265520,22100


In [37]:
from config import *
from utils import *

#정답데이터 기반 제출용 데이터 생성

sql = "select * from stock_price where bse_date between '20230601'and '20230621'"
target_data = execute_sql(sql)
target_data = pd.DataFrame(target_data)
target_data.columns = ['종목코드','일자','시가','고가','저가','종가','조정가','거래량']
target_data2 = target_data[use_col]
target_data2['종목코드'] = 'A' + target_data2['종목코드']
bse_data['일자'] = bse_data['일자'].astype('str')
target_data3 = target_data2[target_data2['일자']=='20230621']

perfect_data = pd.merge(bse_data, target_data3, how='left', on='종목코드')
print(bse_data.shape)
print(perfect_data.shape)
perfect_data['상승률'] = (perfect_data['종가_y'] - perfect_data['종가_x']) / perfect_data['종가_x']
perfect_data2= perfect_data.sort_values(by='상승률',ascending=False)
test = perfect_data2['종목코드']
test = test.reset_index()
test = test.reset_index()
test = test[['종목코드','level_0']]
test.columns = ['종목코드','순위']
test['순위'] = 1 + test['순위']
#정답데이터
test.to_csv('data/sample_submission_20230711.csv',index=False)

,0,1,2,3,4,5,6,7
0,000020,20230601,9610.000000000000,10080.000000000000,9600.000000000000,10080.000000000000,10080.000000000000,239297
1,000020,20230602,10160.000000000000,10190.000000000000,10040.000000000000,10070.000000000000,10070.000000000000,169536
2,000020,20230605,10130.000000000000,10200.000000000000,9870.000000000000,10110.000000000000,10110.000000000000,177016
3,000020,20230607,10120.000000000000,10170.000000000000,9940.000000000000,10010.000000000000,10010.000000000000,141026
4,000020,20230608,9960.000000000000,10310.000000000000,9910.000000000000,10250.000000000000,10250.000000000000,289417
